![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 2.04 Pandas: Transforms and concatenation
### Concatenation
In the previous section we worked with data we created in-session to make two data frames we then merged - joining by columns. In this session we will work in an opposite way: firstly we will read in data from file rather than creating it; then, secondly, we will join datasets via concatenation - joining by rows (shared columns).

To being with we need to access the data. However, we will make things slightly trickier for ourselves by working with an Excel with multiple worksheets. From my.wbs, asynchronous tasks for week 3, you will find a file called "fake_sales.xlsx". The file contains sales data across three locations (the US, UK and Canada) for January and February (one tab for each month). Save it on your computer.

The first task is to move the file to Colab. Run this code to open a file picker to find the file:

In [ ]:
from google.colab import files
upload = files.upload()

We’ll then need to read in this data, and we’ll need to do this one worksheet at a time. Let’s start with January:

In [ ]:
import pandas as pd # if not already done in your session
import numpy as np

jan_df = pd.read_excel('fake_sales.xlsx', sheet_name='January')
jan_df

As shown, we can inspect the data in full to ensure it has imported correctly. However at 31 rows it starts to become a lot of data to review, and in the realworld its likely our data would be much bigger. Let’s use the _head( )_ function to view just the top five rows:

In [ ]:
jan_df.head()

Everything looks fine but let's check the data types/dtypes to be sure:

In [ ]:
print(jan_df.dtypes)

Everything looks as expected! Let's do the same for the 'Feb' worksheet.

In [ ]:
feb_df = pd.read_excel('fake_sales.xlsx', sheet_name='February')
feb_df.head()

We now want to join the data. However, unlike in the previous example we don’t want to join as new columns, but as new rows. This means we use a different type of join than the _merge( )_ we did in the previous session. Instead we will use concatenation – i.e. we will add the February rows to the bottom of the January rows. However, we have one small issue – the column names are not exactly the same. The January file, for instance, shows “US_Sales” while the February file has “US Sales”. To join the two we will need to correct this. We do this with the following code:

In [ ]:
feb_df.columns = ['Date', 'US_Sales', 'UK_Sales', 'Canada_Sales']
feb_df.head()

With this change made we can execute the concatenation code. We are using _pd.concat( )_ (pandas' concatenation function, in which we specify the dataframes to join as a list, the axis we want to join (0 for rows (which we want) or 1 for columns) and the type of join. “Left join” and “right join” are not relevant in this case, as we are joining at the bottom not the sides, but we can choose a join of “inner” to ignore columns not in both sets, or “outer” to include all columns irrespective of whether they are in all the Dataframes. Actually in this case it is irrelevant because the columns are both the same.

In [ ]:
sales_df = pd.concat([jan_df, feb_df], axis=0, join='outer')
sales_df.head()

There are no errors (yay!) but we can only see January data using head - which is unsuprising as our February data is meant to be concatenated to the bottom of the dataframe. Fortunately there is an equivalent function to _head( )_ to show the bottom five records (which should all be February data if everything has worked::

In [ ]:
sales_df.tail()

### Data Transformations
Data transformations cover a wide range of actions including basic cleaning up steps through to advanced mathematical operations. We'll mostly focus on the former here!

Checking back to the Excel sheet we can see that these are blank cells with no data – missing data, "NaNs" or "nulls" are the common name. We can also verify this issues in pandas using the _isnull( )_ function:

In [ ]:
sales_df.isnull().sum()

As we can see there are a total of four null/missing values.

Dealing with missing data is a common problem with no single “right” solution (it depends on the specifics of the case). There is an extensive article on the topic in the pandas documentation (https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html).

In our (fake) scenario, we have followed the most sensible action – we checked with the people who produced the data. They have reported that because of system issues these were actually zero sales days and should be reported as such. We need, therefore to replace the “NaN” records with 0:

In [ ]:
sales_df = sales_df.fillna(0)
sales_df.isnull().sum()

The sum of _isnull( )_ is now zero so everthing has worked.

An additional problem is that we have multiple currencies in our dataframe. From the Excel sheet we see that US and Canada are in dollars (presumably US dollars and Canadian dollars respectively), and UK is in pounds. This will prevent us from comparing these values in a meaningful way. We want to convert all of these columns to the same currency – in this case UK pounds. Checking a currency exchange (https://www.xe.com/) we see that a US dollar is worth £0.76, and a Canadian dollar as £0.56. We need to apply these adjustments to our dataframe.

In [ ]:
sales_df['US_Sales'] = sales_df['US_Sales'] * 0.76
sales_df['Canada_Sales'] = sales_df['Canada_Sales'] * 0.56

Finally, what we want to do with this data is to calculate an average by month for each country. We can use pandas’ _groupby( )_ functionality to do this. However, we first need to establish which data is from February and which is from January (obviously it would have been easier to do this before merging, but for the sake of example). We need to create another calculated field, this time capturing the month:

In [ ]:
sales_df['Month'] = pd.DatetimeIndex(sales_df['Date']).month
sales_df.head()

In [ ]:
sales_df.tail()

The code creates a new column with a pandas DatetimeIndex (an index number related to an aspect of datatime such as a day of the month would be the number and year would be a number), with the final part specifying we want the month index (1 or 2). Again, we can verify this in the notebook.

This transformation achieves what we need, although maybe not in the prettiest way. To fix this up we will use another transformation to change the month record to "Jan" or "Feb":

In [ ]:
sales_df['Month'] = np.where(sales_df.Month == 1, 'Jan', 'Feb')
sales_df.head()

In [ ]:
sales_df.tail()

This command uses numpy (np) – specifically the _where( )_ function that works very much like an if condition. We first apply a logical test (does each item in the "Month" column equal 1) and if True we change it to "Jan" and if False we change it to "Feb". Again this is not necessarily the most efficient way to achieve what we want but it does show some useful functionality.

Finally we do our groupings. We will group by month to get the figures for each region. However, as we aggregate the figures we need to pick an appropriate method. This could be the maximum, the minimum, the median, or so on. In this case we will use the mean average:

In [ ]:
sales_group_df = sales_df.groupby(['Month']).mean()
sales_group_df

As can be seen we now have aggregated/average sales for each day of each month for each of our regions. Good job!

### EXERCISE
1) Re-do the last steps of the sales group process changing the _groupby( )_ function for sales figures to _median( )_ ... what difference does this make to the result? Would this be expected?



### More groupby Options
We use the groupby function when we want to change the unit of aggregation of a DataFrame. For instance, we may have a DataFrame that has 1x row per day and we want to aggregate up to 1x row per month:

In [ ]:
df = pd.DataFrame({"Team": ["DS", "DT", "DT", "DS", "DS"], "salary": [100000, 100, 20000, 0, 100000]})
print("Raw Data")
print(df)
print("\n")

# team based aggregation (average)
# reset index means we convert the output to a flat 2D shape rather than leaving a gap
mean_df = df.groupby(['Team']).mean().reset_index()

print("Aggregation by Mean")
print(mean_df)
print("\n")

# team based aggregation (sum)
sum_df = df.groupby(['Team']).sum().reset_index()

print("Aggregation by Sum")
print(sum_df)
print("\n")

# team based aggregation (median)
median_df = df.groupby(['Team']).median().reset_index()

print("Aggregation by Median")
print(median_df)
print("\n")

# team based aggregation (maximum)
max_df = df.groupby(['Team']).max().reset_index()

print("Aggregation by Maximum")
print(max_df)
print("\n")

### Time and Date
In this last part we will use pandas' date/time functionality for data  transforms:

In [ ]:
df = pd.DataFrame({"id": [3, 1, 2, 4, 10], "salary": [100000, 100, 20000, 0, 100000],
                   "dob": ["10/01/1950", "01/01/1990", "01/01/1980", "12/12/1981", "13/06/1981"]})
print(df)

# convert a column which has a date stored as string, into a column
# stored as date. dayfirst=True means we use the UK style of date
# (DDMMYYYY) rather than US (MMDDYYYY)
df['dob'] = pd.to_datetime(df['dob'], dayfirst=True)

# from a date object create cols for year, month, day, hour and minute
df['Year'] =  pd.DatetimeIndex(df['dob']).year
df['Month'] = pd.DatetimeIndex(df['dob']).month
df['Day'] = pd.DatetimeIndex(df['dob']).day
df['Hour'] = pd.DatetimeIndex(df['dob']).hour
df['Minute'] = pd.DatetimeIndex(df['dob']).minute

print("\n")
print(df)

# work out age by substracting the DOB from the date today
# note this is much harder than you may think it would be because years
# are different lengths and so ambiguous
df["Age"] = pd.to_datetime("today", dayfirst=True)-df['dob']
df["Age"] = round(df.Age.dt.days / 365, 1)

print("\n")
print(df)

As an additional example:

In [ ]:
df = pd.DataFrame({"student": [1, 2, 3, 4, 5],
                   "deadline": ["01/11/2025", "02/11/2025", "03/11/2025", "04/11/2025", "05/11/2025"],
                   "submission": ["01/11/2025", "01/11/2025", "02/12/2025", "03/11/2025", "01/12/2025"]})

df.head()

In [ ]:
# convert a column which has a date stored as string, into a column
# stored as date. dayfirst=True means we use the UK style of date
# (DDMMYYYY) rather than US (MMDDYYYY)
# but this time inside a for loop

date_cols = ["deadline", "submission"]

for col in date_cols:
  df[col] = pd.to_datetime(df[col], dayfirst=True)

df.head()

Now let's look at when students submitted vs deadline:

In [ ]:
df["after_deadline"] = df["submission"] - df["deadline"]
df.head()

__PUBLIC SERVICE ANNOUNCEMENT__: Don't submit after the deadline ... it's very expensive.